In [11]:
!pip install -q aiohttp beautifulsoup4 lxml pandas nest_asyncio tqdm ipywidgets


In [12]:

import nest_asyncio
nest_asyncio.apply()


import asyncio
import time
from urllib.parse import urlparse
from urllib.robotparser import RobotFileParser

import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import clear_output, display
from google.colab import files

In [13]:
print("Upload a file named urls.txt (one URL per line).")
uploaded = files.upload()

import os
if 'urls.txt' in uploaded:
    with open('/content/urls.txt', 'wb') as f:
        f.write(uploaded['urls.txt'])
    urls_file = '/content/urls.txt'
else:

    txts = [name for name in uploaded if name.lower().endswith('.txt')]
    if txts:
        with open('/content/urls.txt', 'wb') as f:
            f.write(uploaded[txts[0]])
        urls_file = '/content/urls.txt'
    else:
        raise FileNotFoundError("Please upload a urls.txt file with one URL per line.")

with open(urls_file, 'r') as f:
    urls = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(urls)} URLs. Example first 5:\n", urls[:5])

Upload a file named urls.txt (one URL per line).


Saving urls_10000.txt to urls_10000.txt
Loaded 10000 URLs. Example first 5:
 ['https://example.com?id=0', 'https://httpbin.org?id=1', 'https://www.wikipedia.org?id=2', 'https://www.python.org?id=3', 'https://www.openai.com?id=4']


In [14]:
async def fetch_robots(session, base_url, cache):
    """
    Returns a RobotFileParser instance for the domain (cached).
    base_url example: 'https://example.com'
    """
    domain = urlparse(base_url).netloc
    if domain in cache:
        return cache[domain]

    robots_url = f"{urlparse(base_url).scheme}://{domain}/robots.txt"
    rp = RobotFileParser()
    try:
        async with session.get(robots_url) as resp:
            if resp.status == 200:
                raw = await resp.read()
                try:
                    text = raw.decode('utf-8', errors='ignore')
                except:
                    text = raw.decode('latin1', errors='ignore')
                rp.parse(text.splitlines())
            else:

                rp.parse([])
    except Exception:

        rp.parse([])

    cache[domain] = rp
    return rp



async def fetch_one(session, url, sem, robots_cache, user_agent="real-time-scraper/1.0"):
    async with sem:
        start = time.time()
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"

        try:
            rp = await fetch_robots(session, base, robots_cache)
            allowed = rp.can_fetch("*", url)
        except Exception:
            allowed = True

        if not allowed:
            return {"url": url, "status": "blocked_by_robots", "title": None, "meta_description": None, "h1": None, "elapsed": 0.0}

        headers = {"User-Agent": user_agent}
        try:
            async with session.get(url, headers=headers) as resp:
                status = resp.status
                raw = await resp.read()
                try:
                    text = raw.decode('utf-8', errors='ignore')
                except:
                    text = raw.decode('latin1', errors='ignore')
                elapsed = time.time() - start


                soup = BeautifulSoup(text, "lxml")
                title = soup.title.string.strip() if soup.title and soup.title.string else None
                meta = soup.find("meta", attrs={"name": "description"})
                meta_desc = meta["content"].strip() if meta and meta.get("content") else None
                h1 = soup.find("h1")
                h1_text = h1.get_text(strip=True) if h1 else None

                return {"url": url, "status": status, "title": title, "meta_description": meta_desc, "h1": h1_text, "elapsed": elapsed}
        except Exception as e:
            return {"url": url, "status": f"error:{type(e)._name_}", "title": None, "meta_description": None, "h1": None, "elapsed": 0.0}

In [15]:
async def worker(name, session, queue, results, sem, robots_cache, display_every, save_path):
    while True:
        url = await queue.get()
        if url is None:
            queue.task_done()
            break
        res = await fetch_one(session, url, sem, robots_cache)
        results.append(res)

        if len(results) % display_every == 0:
            df = pd.DataFrame(results)
            df.to_csv(save_path, index=False)
            clear_output(wait=True)
            print(f"Scraped: {len(results)} / {queue.total_items} (in progress)")
            display(df.tail(10))
        queue.task_done()


async def run_scraper(urls, concurrency=50, display_every=50, save_path="/content/results.csv"):
    queue = asyncio.Queue()
    queue.total_items = len(urls)
    for u in urls:
        await queue.put(u)

    sem = asyncio.Semaphore(concurrency)
    results = []
    robots_cache = {}

    timeout = aiohttp.ClientTimeout(total=30)
    conn = aiohttp.TCPConnector(limit_per_host=10)

    async with aiohttp.ClientSession(timeout=timeout, connector=conn) as session:
        workers = []
        for i in range(concurrency):
            w = asyncio.create_task(worker(f"worker-{i}", session, queue, results, sem, robots_cache, display_every, save_path))
            workers.append(w)

        await queue.join()

        for _ in workers:
            await queue.put(None)
        await asyncio.gather(*workers)


    df = pd.DataFrame(results)
    df.to_csv(save_path, index=False)
    return df

In [16]:
CONCURRENCY = 50
DISPLAY_EVERY = 50
SAVE_PATH = "/content/results.csv"

print("Starting scraper with concurrency =", CONCURRENCY)

df = asyncio.get_event_loop().run_until_complete(run_scraper(urls, concurrency=CONCURRENCY, display_every=DISPLAY_EVERY, save_path=SAVE_PATH))

print("Scraping finished. Final results saved to:", SAVE_PATH)
display(df.head())

Scraped: 10000 / 10000 (in progress)


url  status             title  \
9990  https://flask.palletsprojects.com?id=9959     403  Just a moment...   
9991  https://flask.palletsprojects.com?id=9899     403  Just a moment...   
9992  https://flask.palletsprojects.com?id=9969     403  Just a moment...   
9993             https://www.openai.com?id=9984     403              None   
9994  https://flask.palletsprojects.com?id=9989     403  Just a moment...   
9995  https://flask.palletsprojects.com?id=9999     403  Just a moment...   
9996      https://www.stackoverflow.com?id=9976     429  Just a moment...   
9997  https://flask.palletsprojects.com?id=9979     403  Just a moment...   
9998      https://www.stackoverflow.com?id=9966     429  Just a moment...   
9999             https://www.openai.com?id=9994     403              None   

     meta_description    h1   elapsed  
9990             None  None  2.831192  
9991             None  None  4.676327  
9992             None  None  2.663977  
9993             None  None  2.004173  
9994             None  None  1.616874  
9995             None  None  1.353602  
9996             None  None  2.121386  
9997             None  None  2.097834  
9998             None  None  2.772690  
9999             None  None  1.553518

Scraping finished. Final results saved to: /content/results.csv


url  status                            title  \
0        https://pypi.org?id=45     200  PyPI · The Python Package Index   
1        https://pypi.org?id=15     200  PyPI · The Python Package Index   
2  https://www.python.org?id=43     200            Welcome to Python.org   
3   https://www.python.org?id=3     200            Welcome to Python.org   
4  https://www.python.org?id=13     200            Welcome to Python.org   

                                    meta_description  \
0  The Python Package Index (PyPI) is a repositor...   
1  The Python Package Index (PyPI) is a repositor...   
2  The official home of the Python Programming La...   
3  The official home of the Python Programming La...   
4  The official home of the Python Programming La...   

                                                  h1   elapsed  
0  Find, install and publish Python packages with...  0.137687  
1  Find, install and publish Python packages with...  0.151367  
2                                                     0.165656  
3                                                     0.198060  
4                                                     0.216298

from matplotlib import pyplot as plt
_df_20['elapsed'].plot(kind='hist', bins=20, title='elapsed')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_34['title'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_34, x='elapsed', y='title', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_35['meta_description'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_35, x='elapsed', y='meta_description', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_36['h1'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_36, x='elapsed', y='h1', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)